# INITIAL NETWORK INFERENCE STEP

In [1]:
import numpy as np
from scipy.stats import entropy
from itertools import combinations
from sklearn.metrics import mutual_info_score

### DATA GENERATION - GT NETWORK SIM

v0 = v1 AND NOT v2 <br>
v1 = v0 OR v3 <br>
v2 = NOT v4 <br>
v3 = v2 OR NOT v5 <br>
v4 = v3 AND v6 <br>
v5 = NOT v1 OR v4 <br>
v6 = v5 AND NOT v0

In [2]:
import numpy as np

# ──────────────────────────────────────────────────────────────────────────────
# 1) Define the update rule for each of the 7 nodes
#    We use only the logical relations AND, OR, NOT (and their combinations
#    like AND NOT, OR NOT) across the network.
# ──────────────────────────────────────────────────────────────────────────────

def update_7(state):
    """
    state: 1D array of length 7 with values 0 or 1
      indices: [v0, v1, v2, v3, v4, v5, v6]
    returns: new_state, same shape
    """
    v0, v1, v2, v3, v4, v5, v6 = state

    # v0(t+1) = v1 AND (NOT v2)
    new0 = int(v1 and not v2)

    # v1(t+1) = v0 OR v3
    new1 = int(v0 or v3)

    # v2(t+1) = NOT v4
    new2 = int(not v4)

    # v3(t+1) = v2 OR (NOT v5)         # OR NOT
    new3 = int(v2 or not v5)

    # v4(t+1) = v3 AND v6             # AND
    new4 = int(v3 and v6)

    # v5(t+1) = (NOT v1) OR v4         # OR NOT
    new5 = int((not v1) or v4)

    # v6(t+1) = v5 AND (NOT v0)        # AND NOT
    new6 = int(v5 and not v0)

    return np.array([new0, new1, new2, new3, new4, new5, new6], dtype=int)


# ──────────────────────────────────────────────────────────────────────────────
# 2) Simulator
# ──────────────────────────────────────────────────────────────────────────────

def simulate_7(initial_state, steps=50):
    """
    initial_state: array‐like of length 7 (0/1)
    steps: number of synchronous update steps
    returns: history, a (steps × 7) array of 0/1
    """
    history = np.zeros((steps, 7), dtype=int)
    state = np.array(initial_state, dtype=int)

    for t in range(steps):
        history[t] = state
        state = update_7(state)

    return history


# ──────────────────────────────────────────────────────────────────────────────
# 3) Example usage
# ──────────────────────────────────────────────────────────────────────────────

if __name__ == "__main__":
    # pick a random initial state for the 7‐node network
    init = np.random.randint(0, 2, size=7)
    traj = np.array(simulate_7(init, steps=50))

    print("Initial state:", init)
    print("Trajectory shape:", traj.shape)   # should be (50,7)
    print("Trajectory:")
    print(traj)  # each row is [v0,v1,…,v6] at that time step


Initial state: [0 1 0 1 1 1 0]
Trajectory shape: (50, 7)
Trajectory:
[[0 1 0 1 1 1 0]
 [1 1 0 0 0 1 1]
 [1 1 1 0 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]
 [0 1 1 1 0 0 0]]


## MIBNI Network Inference

In [3]:
import numpy as np
import math
from itertools import product
from collections import Counter

# ──────────────────────────────────────────────────────────────────────────────
# Entropy / MI routines (unchanged)
# ──────────────────────────────────────────────────────────────────────────────

def entropy(arr):
    n = len(arr)
    counts = Counter(arr)
    H = 0.0
    for cnt in counts.values():
        p = cnt / n
        H -= p * math.log2(p)
    return H

def joint_entropy(arrays):
    joint = list(zip(*arrays))
    counts = Counter(joint)
    H = 0.0
    n = sum(counts.values())
    for cnt in counts.values():
        p = cnt / n
        H -= p * math.log2(p)
    return H

def mutual_information(x, y):
    return entropy(x) + entropy(y) - joint_entropy([x, y])

def conditional_mutual_information(x, z, cond):
    if isinstance(cond, np.ndarray):
        cond = [cond]
    H_xc  = joint_entropy([x, *cond])
    H_cz  = joint_entropy([*cond, z])
    H_c   = joint_entropy(cond)
    H_xcz = joint_entropy([x, *cond, z])
    return H_xc + H_cz - H_c - H_xcz

# ──────────────────────────────────────────────────────────────────────────────
# MIBNI regulator selection (unchanged)
# ──────────────────────────────────────────────────────────────────────────────

def compute_dynamic_consistency(y, S_idx, data):
    if len(S_idx)==0:
        maj = Counter(y).most_common(1)[0][0]
        return sum(y==maj)/len(y), {(): maj}
    pats = data[:, S_idx]
    table = {}
    for pat in {tuple(r) for r in pats}:
        mask = np.all(pats == pat, axis=1)
        maj  = Counter(y[mask]).most_common(1)[0][0]
        table[pat] = maj
    preds = np.array([table[tuple(r)] for r in pats])
    return np.mean(preds==y), table

def MIFS(y, W_idx, data, k):
    S = []
    W = W_idx.copy()
    scores = {w: mutual_information(y, data[:,w]) for w in W}
    best = max(scores, key=scores.get)
    S.append(best); W.remove(best)
    while len(S)<k and W:
        scores = { w: conditional_mutual_information(y, data[:,w], [data[:,s] for s in S]) for w in W }
        best = max(scores, key=scores.get)
        S.append(best); W.remove(best)
    return S

def SWAP(y, S_idx, W_idx, data):
    E_max, _ = compute_dynamic_consistency(y, S_idx, data)
    best_S, best_E = S_idx[:], E_max
    improved = True
    while improved:
        improved = False
        for si in list(best_S):
            for wj in W_idx:
                S_new = [x for x in best_S if x!=si] + [wj]
                E_new, _ = compute_dynamic_consistency(y, S_new, data)
                if E_new > best_E + 1e-9:
                    best_S, best_E = S_new, E_new
                    W_idx = [x for x in W_idx if x!=wj] + [si]
                    improved = True
                    break
            if improved: break
    return best_S, best_E

# ──────────────────────────────────────────────────────────────────────────────
# Allowed‐logic fitting (fixed constant functions)
# ──────────────────────────────────────────────────────────────────────────────

def fit_allowed_logic(y, pats):
    """
    Given target y (n,) and pats = data[:, regs] (n,m), m<=2,
    return (best_expr, best_fn, best_accuracy).
    """
    n, m = pats.shape
    best = ("0", lambda *args: np.zeros(n, dtype=int), -1.0)

    # constants 0 and 1
    for c in [0,1]:
        fn = (lambda *args, c=c: np.full(n, c, dtype=int))
        acc = np.mean(y == c)
        if acc > best[2]:
            best = (str(c), fn, acc)

    # unary
    if m == 1:
        a = pats[:, 0]
        for expr, fn in [("v",        (lambda a: a)),
                         ("NOT v",    (lambda a: 1 - a))]:
            preds = fn(a)
            acc   = np.mean(preds == y)
            if acc > best[2]:
                best = (expr, fn, acc)

    # binary
    if m == 2:
        a, b = pats[:,0], pats[:,1]
        candidates = [
            ("v1 AND v2",     (lambda a,b: a & b)),
            ("v1 OR v2",      (lambda a,b: a | b)),
            ("v1 AND NOT v2", (lambda a,b: a & (1 - b))),
            ("NOT v1 AND v2", (lambda a,b: (1 - a) & b)),
            ("v1 OR NOT v2",  (lambda a,b: a | (1 - b))),
            ("NOT v1 OR v2",  (lambda a,b: (1 - a) | b)),
        ]
        for expr, fn in candidates:
            preds = fn(a, b)
            acc   = np.mean(preds == y)
            if acc > best[2]:
                best = (expr, fn, acc)

    return best

# ──────────────────────────────────────────────────────────────────────────────
# Main inference (K=2) + pretty‐print
# ──────────────────────────────────────────────────────────────────────────────

def infer_boolean_network_restricted(data):
    n_samples, n_vars = data.shape
    K = 2
    net = {}

    for i in range(n_vars):
        y = data[:,i]
        if entropy(y)==0:
            net[i] = {"regs": [], "expr":"%d"%y[0], "acc":1.0}
            continue

        W = [j for j in range(n_vars) if j!=i]
        for k in range(1,K+1):
            S = MIFS(y, W, data, k)
            S_swapped,_ = SWAP(y, S, [w for w in W if w not in S], data)
            pats = data[:, S_swapped]
            expr, fn, acc = fit_allowed_logic(y, pats)
            # remap v1/v2/v tokens
            if expr not in ("0","1"):
                if len(S_swapped)==1:
                    expr = expr.replace("v", f"v{S_swapped[0]}")
                else:
                    expr = expr.replace("v1", f"v{S_swapped[0]}")
                    expr = expr.replace("v2", f"v{S_swapped[1]}")
            if acc>=1-1e-9 or k==K:
                net[i] = {"regs":S_swapped, "expr":expr, "acc":acc}
                break

    # pretty‐print
    for i in sorted(net):
        entry = net[i]
        print(f"v{i} = {entry['expr']}   ")

    return net

# ──────────────────────────────────────────────────────────────────────────────
# Example on your 7‐node simulated data
# ──────────────────────────────────────────────────────────────────────────────

if __name__ == "__main__":
    def update_7(s):
        v0,v1,v2,v3,v4,v5,v6 = s
        return np.array([
            v1 and not v2,
            v0 or v3,
            not v4,
            v2 or not v5,
            v3 and v6,
            not v1 or v4,
            v5 and not v0
        ],dtype=int)

    state = np.random.randint(0,2,7)
    traj  = np.zeros((50,7), int)
    for t in range(50):
        traj[t] = state
        state = update_7(state)

    infer_boolean_network_restricted(traj)
    


v0 = v6 AND NOT v5   
v1 = NOT v0 OR v2   
v2 = NOT v5 AND v1   
v3 = NOT v0 AND v1   
v4 = v5 AND v2   
v5 = 0   
v6 = v0 OR NOT v2   


## DECISION TREE CLASSIFIER INFERENCE

In [4]:
import numpy as np
from itertools import product
from sklearn.tree import DecisionTreeClassifier

def infer_boolean_network_dt_unrestrained(data, max_depth=None):
    """
    Unrestricted decision-tree inference of a Boolean network.

    Parameters
    ----------
    data : ndarray, shape (n_samples, n_vars)
        Binary (0/1) expression states.
    max_depth : int or None
        Passed to DecisionTreeClassifier; None grows until pure leaves.

    Returns
    -------
    network : dict
      target_idx -> {
        'regs':        [j,k,…],                 # regulator indices
        'truth_table': {pattern:bit,…},         # full lookup
        'accuracy':    float                    # training accuracy (==1.0)
      }
    """
    n_samples, n_vars = data.shape
    network = {}

    for i in range(n_vars):
        # prepare features/target
        X = np.delete(data, i, axis=1)
        y = data[:, i]

        # train unrestricted tree
        clf = DecisionTreeClassifier(max_depth=max_depth)
        clf.fit(X, y)

        # find which feature columns were used in the splits
        used_feats = sorted({f for f in clf.tree_.feature if f >= 0})
        # map back to original variable indices
        regs = [(f if f < i else f + 1) for f in used_feats]

        # build the exact truth table over those regulators
        truth_table = {}
        for pattern in product([0, 1], repeat=len(regs)):
            # build a single‐row input to predict()
            row = np.zeros((1, n_vars - 1), dtype=int)
            for bit, orig_idx in zip(pattern, regs):
                feat_idx = orig_idx if orig_idx < i else orig_idx - 1
                row[0, feat_idx] = bit
            truth_table[pattern] = int(clf.predict(row)[0])

        # compute accuracy on the original data
        preds = clf.predict(X)
        accuracy = float(np.mean(preds == y))

        network[i] = {
            'regs': regs,
            'truth_table': truth_table,
            'accuracy': accuracy
        }

    return network

def pretty_print_network(network):
    """
    Given the network dict from above, print each node as:
      v0 = (NOT v2 AND v1) OR (v2 AND NOT v1) OR …
    """
    for i in sorted(network):
        info = network[i]
        regs = info['regs']
        table = info['truth_table']

        # collect all minterms mapping to 1
        terms = []
        for pat, out in table.items():
            if out == 1:
                lits = []
                for bit, r in zip(pat, regs):
                    if bit:
                        lits.append(f"v{r}")
                    else:
                        lits.append(f"NOT v{r}")
                terms.append("(" + " AND ".join(lits) + ")")

        expr = " OR ".join(terms) if terms else "0"
        print(f"v{i} = {expr}    ")

# ──────────────────────────────────────────────────────────────────────────────
# Example usage on your 7-node trajectory `traj`:
# ──────────────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    # assume `traj` is (T×7) ground-truth data you already have
    net_dt_unres = infer_boolean_network_dt_unrestrained(traj, max_depth=None)
    pretty_print_network(net_dt_unres)


v0 = (NOT v5 AND v6)    
v1 = (NOT v2 AND v5) OR (v2 AND NOT v5) OR (v2 AND v5)    
v2 = (NOT v1 AND v4 AND v5) OR (v1 AND NOT v4 AND NOT v5) OR (v1 AND v4 AND NOT v5) OR (v1 AND v4 AND v5)    
v3 = (NOT v0 AND NOT v1) OR (NOT v0 AND v1) OR (v0 AND NOT v1)    
v4 = (v2 AND v3 AND v5)    
v5 = (NOT v0 AND NOT v2 AND NOT v4) OR (NOT v0 AND NOT v2 AND v4) OR (NOT v0 AND v2 AND v4)    
v6 = (v0 AND NOT v2) OR (v0 AND v2)    


In [5]:
gt_network = '''v0 = v1 AND NOT v2 
    v1 = v0 OR v3 
    v2 = NOT v4 
    v3 = v2 OR NOT v5 
    v4 = v3 AND v6 
    v5 = NOT v1 OR v4 
    v6 = v5 AND NOT v0'''

mibni_network = '''v0 = v3 AND NOT v3   
    v1 = 1   
    v2 = NOT v0 AND v3   
    v3 = NOT v0 OR v5   
    v4 = v6   
    v5 = NOT v3 AND v3   
    v6 = v4  '''

dt_network = '''v0 = (NOT v3)    
v1 = ()    
v2 = (NOT v5)    
v3 = (NOT v0)    
v4 = 0    
v5 = (NOT v2 AND NOT v4 AND NOT v6) OR (NOT v2 AND NOT v4 AND v6) OR (NOT v2 AND v4 AND NOT v6) OR (NOT v2 AND v4 AND v6)    
v6 = (NOT v3 AND NOT v4 AND NOT v5) OR (NOT v3 AND NOT v4 AND v5) OR (NOT v3 AND v4 AND NOT v5) OR (NOT v3 AND v4 AND v5) '''


In [6]:
import re
import numpy as np
from itertools import product

# ──────────────────────────────────────────────────────────────────────────────
# 1) Your network strings
# ──────────────────────────────────────────────────────────────────────────────
gt_network = '''v0 = v1 AND NOT v2 
v1 = v0 OR v3 
v2 = NOT v4 
v3 = v2 OR NOT v5 
v4 = v3 AND v6 
v5 = NOT v1 OR v4 
v6 = v5 AND NOT v0'''

mibni_network = '''v0 = v3 AND NOT v3   
v1 = 1   
v2 = NOT v0 AND v3   
v3 = NOT v0 OR v5   
v4 = v6   
v5 = NOT v3 AND v3   
v6 = v4'''

dt_network = '''v0 = (NOT v3)    
v1 = ()    
v2 = (NOT v5)    
v3 = (NOT v0)    
v4 = 0    
v5 = (NOT v2 AND NOT v4 AND NOT v6) OR (NOT v2 AND NOT v4 AND v6) OR (NOT v2 AND v4 AND NOT v6) OR (NOT v2 AND v4 AND v6)    
v6 = (NOT v3 AND NOT v4 AND NOT v5) OR (NOT v3 AND NOT v4 AND v5) OR (NOT v3 AND v4 AND NOT v5) OR (NOT v3 AND v4 AND v5) '''

# ──────────────────────────────────────────────────────────────────────────────
# 2) Parser: turn string → dict with 'regs' and 'truth_table'
# ──────────────────────────────────────────────────────────────────────────────
def parse_network(network_str):
    net = {}
    lines = network_str.strip().splitlines()
    for line in lines:
        line = line.strip()
        if not line: 
            continue
        left, right = line.split('=', 1)
        var = left.strip()
        i = int(re.match(r'v(\d+)', var).group(1))
        expr_str = right.strip()
        # find regulators
        regs = sorted({int(n) for n in re.findall(r'v(\d+)', expr_str)})
        # prepare Python‐style boolean expression
        expr_py = expr_str.replace('AND', 'and').replace('OR', 'or').replace('NOT', 'not')
        # build truth table
        truth_table = {}
        for pattern in product([0,1], repeat=len(regs)):
            env = {f'v{r}': pattern[idx] for idx, r in enumerate(regs)}
            val = eval(expr_py, {}, env)
            truth_table[tuple(pattern)] = int(bool(val))
        net[i] = {
            'regs': regs,
            'truth_table': truth_table
        }
    return net

# parse all three
net_true  = parse_network(gt_network)
net_mibni = parse_network(mibni_network)
net_dt    = parse_network(dt_network)

# ──────────────────────────────────────────────────────────────────────────────
# 3) Topology similarity
# ──────────────────────────────────────────────────────────────────────────────
def topology_similarity(net_true, net_pred):
    edges_true = {(j,i) for i,info in net_true.items() for j in info['regs']}
    edges_pred = {(j,i) for i,info in net_pred.items() for j in info['regs']}
    TP = edges_true & edges_pred
    prec = len(TP)/len(edges_pred) if edges_pred else 0.0
    rec  = len(TP)/len(edges_true)  if edges_true  else 0.0
    f1   = 2*prec*rec/(prec+rec)     if (prec+rec)>0 else 0.0
    # per‐node Jaccard
    per_j = {}
    for i in net_true:
        S_true = set(net_true[i]['regs'])
        S_pred = set(net_pred[i]['regs'])
        U = S_true | S_pred
        per_j[i] = len(S_true & S_pred)/len(U) if U else 1.0
    return {
        'precision': prec,
        'recall':    rec,
        'f1':        f1,
        'avg_jaccard': np.mean(list(per_j.values())),
        'per_jaccard': per_j
    }

# compute topology scores
topo_mibni = topology_similarity(net_true, net_mibni)
topo_dt    = topology_similarity(net_true, net_dt)

print("=== Topology similarity ===")
print("MIBNI:", topo_mibni)
print("   DT:", topo_dt)

# ──────────────────────────────────────────────────────────────────────────────
# 4) Trajectory similarity
#    assumes you have a NumPy array `traj` of shape (T, n_vars)
# ──────────────────────────────────────────────────────────────────────────────
def make_update_fn(network, n_vars):
    def update(state):
        new = np.zeros(n_vars, dtype=int)
        for i, info in network.items():
            regs = info['regs']
            if regs:
                pat = tuple(state[j] for j in regs)
                new[i] = info['truth_table'][pat]
            else:
                new[i] = next(iter(info['truth_table'].values()))
        return new
    return update

def simulate_network(update_fn, initial_state, steps):
    n_vars = len(initial_state)
    traj_pred = np.zeros((steps, n_vars), dtype=int)
    state = initial_state.copy()
    for t in range(steps):
        traj_pred[t] = state
        state = update_fn(state)
    return traj_pred

def trajectory_similarity(traj_true, traj_pred):
    return np.mean(traj_true == traj_pred)

# run trajectory comparison
init   = traj[0]
steps  = traj.shape[0]
n_vars = traj.shape[1]

upd_mibni = make_update_fn(net_mibni, n_vars)
upd_dt    = make_update_fn(net_dt,    n_vars)

traj_mibni = simulate_network(upd_mibni, init, steps)
traj_dt    = simulate_network(upd_dt,    init, steps)

sim_mibni = trajectory_similarity(traj, traj_mibni)
sim_dt    = trajectory_similarity(traj, traj_dt)

print("\n=== Trajectory similarity (fraction matching) ===")
print(f"MIBNI vs GT: {sim_mibni:.3f}")
print(f"DT   vs GT: {sim_dt:.3f}")


=== Topology similarity ===
MIBNI: {'precision': 0.25, 'recall': 0.15384615384615385, 'f1': 0.1904761904761905, 'avg_jaccard': 0.11904761904761904, 'per_jaccard': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.3333333333333333, 4: 0.5, 5: 0.0, 6: 0.0}}
   DT: {'precision': 0.2222222222222222, 'recall': 0.15384615384615385, 'f1': 0.18181818181818185, 'avg_jaccard': 0.07142857142857142, 'per_jaccard': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.25, 6: 0.25}}

=== Trajectory similarity (fraction matching) ===
MIBNI vs GT: 0.834
DT   vs GT: 0.506


In [7]:
import numpy as np
from itertools import product
from sklearn.tree import DecisionTreeClassifier

# (paste in the infer_boolean_network_dt_unrestrained and make_update_fn,
#  topology_similarity, simulate_network, trajectory_similarity from before)

# 1) Generate your GT trajectory `traj` as you already have:
#    traj.shape == (T, 7)

# 2) Parse the GT string into net_true
net_true = parse_network(gt_network)   # your ground‐truth parser

# 3) Infer the **unrestrained** DT network directly from the raw traj data:
net_dt_unres = infer_boolean_network_dt_unrestrained(traj, max_depth=None)

# 4) Check that every node was fit perfectly on training:
for i,info in net_dt_unres.items():
    print(f"node v{i} train‐accuracy = {info['accuracy']:.3f}")
# You should see 1.000 for all i.

# 5) Now recompute similarities **directly** on the dicts:
topo_dt    = topology_similarity(net_true, net_dt_unres)

# 6) Simulate from the same initial state:
init   = traj[0]
steps  = traj.shape[0]
n_vars = traj.shape[1]

upd_dt    = make_update_fn(net_dt_unres, n_vars)
traj_dt   = simulate_network(upd_dt, init, steps)

sim_dt    = trajectory_similarity(traj, traj_dt)

print("=== Topology similarity (unrestr DT) ===", topo_dt)
print("=== Trajectory similarity (unrestr DT) ===", sim_dt)


node v0 train‐accuracy = 1.000
node v1 train‐accuracy = 1.000
node v2 train‐accuracy = 1.000
node v3 train‐accuracy = 1.000
node v4 train‐accuracy = 0.980
node v5 train‐accuracy = 1.000
node v6 train‐accuracy = 0.980
=== Topology similarity (unrestr DT) === {'precision': 0.3333333333333333, 'recall': 0.46153846153846156, 'f1': 0.3870967741935484, 'avg_jaccard': 0.24999999999999997, 'per_jaccard': {0: 0.0, 1: 0.3333333333333333, 2: 0.3333333333333333, 3: 0.0, 4: 0.25, 5: 0.5, 6: 0.3333333333333333}}
=== Trajectory similarity (unrestr DT) === 0.44


In [8]:
import numpy as np

# find the first mismatch
mm = np.argwhere(traj != traj_dt)
if mm.size == 0:
    print("Perfect match!")
else:
    t0, node = mm[0]
    print(f"First mismatch at step {t0}, node v{node}:")
    print("  GT:", traj[t0,node], "  DT:", traj_dt[t0,node])

    # see the regulator‐pattern that caused it
    regs = net_dt_unres[node]['regs']
    prev_state = traj_dt[t0-1]
    pat = tuple(prev_state[j] for j in regs)
    print("  Regulators for v{}: {}".format(node, regs))
    print("  Pattern at t-1:", pat)
    print("  DT lookup gives:", net_dt_unres[node]['truth_table'][pat])


First mismatch at step 1, node v0:
  GT: 0   DT: 1
  Regulators for v0: [5, 6]
  Pattern at t-1: (0, 1)
  DT lookup gives: 1
